In [2]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

# Melbourne Housing Market #

## Melbourne housing clearance data from Jan 2016 ##

### Content & Acknowledgements ###

This data was scraped from publicly available results posted every week from Domain.com.au, I've cleaned it as best I can, now it's up to you to make data analysis magic. The dataset includes Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale and distance from C.B.D.

## Attribute Information ##

Suburb: Suburb

Address: Address

Rooms: Number of rooms

Price: Price in Australian dollars

Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.

Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.

SellerG: Real Estate Agent

Date: Date sold

Distance: Distance from CBD

Regionname: General Region (West, North West, North, North east ...etc)

Propertycount: Number of properties that exist in the suburb.

Bedroom2 : Scraped # of Bedrooms (from different source)

Bathroom: Number of Bathrooms

Car: Number of carspots

Landsize: Land Size

BuildingArea: Building Size

YearBuilt: Year the house was built

CouncilArea: Governing council for the area

Lattitude: Self explanatory

Longtitude: Self explanatory

In [3]:
houses = pd.read_csv(("../data/Melbourne_housing_FULL.csv"))

In [4]:
houses.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,nan,SS,Jellis,3/09/2016,2.500,3067.000,...,1.000,1.000,126.000,nan,nan,Yarra City Council,-37.801,144.996,Northern Metropolitan,4019.000
1,Abbotsford,85 Turner St,2,h,1480000.000,S,Biggin,3/12/2016,2.500,3067.000,...,1.000,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000
2,Abbotsford,25 Bloomburg St,2,h,1035000.000,S,Biggin,4/02/2016,2.500,3067.000,...,1.000,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000
3,Abbotsford,18/659 Victoria St,3,u,nan,VB,Rounds,4/02/2016,2.500,3067.000,...,2.000,1.000,0.000,nan,nan,Yarra City Council,-37.811,145.012,Northern Metropolitan,4019.000
4,Abbotsford,5 Charles St,3,h,1465000.000,SP,Biggin,4/03/2017,2.500,3067.000,...,2.000,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000


In [5]:
houses.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,34857.000,27247.000,34856.000,34856.000,26640.000,26631.000,26129.000,23047.000,13742.000,15551.000,26881.000,26881.000,34854.000
mean,3.031,1050173.345,11.185,3116.063,3.085,1.625,1.729,593.599,160.256,1965.290,-37.811,145.002,7572.888
std,0.970,641467.130,6.789,109.024,0.981,0.724,1.011,3398.842,401.267,37.328,0.090,0.120,4428.090
min,1.000,85000.000,0.000,3000.000,0.000,0.000,0.000,0.000,0.000,1196.000,-38.190,144.424,83.000
25%,2.000,635000.000,6.400,3051.000,2.000,1.000,1.000,224.000,102.000,1940.000,-37.863,144.934,4385.000
50%,3.000,870000.000,10.300,3103.000,3.000,2.000,2.000,521.000,136.000,1970.000,-37.808,145.008,6763.000
75%,4.000,1295000.000,14.000,3156.000,4.000,2.000,2.000,670.000,188.000,2000.000,-37.754,145.072,10412.000
max,16.000,11200000.000,48.100,3978.000,30.000,12.000,26.000,433014.000,44515.000,2106.000,-37.390,145.526,21650.000


In [12]:
houses = houses[houses["Price"].notnull()]

In [13]:
houses.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,1480000.000,S,Biggin,3/12/2016,2.500,3067.000,...,1.000,1.000,202.000,nan,nan,Yarra City Council,-37.800,144.998,Northern Metropolitan,4019.000
2,Abbotsford,25 Bloomburg St,2,h,1035000.000,S,Biggin,4/02/2016,2.500,3067.000,...,1.000,0.000,156.000,79.000,1900.000,Yarra City Council,-37.808,144.993,Northern Metropolitan,4019.000
4,Abbotsford,5 Charles St,3,h,1465000.000,SP,Biggin,4/03/2017,2.500,3067.000,...,2.000,0.000,134.000,150.000,1900.000,Yarra City Council,-37.809,144.994,Northern Metropolitan,4019.000
5,Abbotsford,40 Federation La,3,h,850000.000,PI,Biggin,4/03/2017,2.500,3067.000,...,2.000,1.000,94.000,nan,nan,Yarra City Council,-37.797,144.997,Northern Metropolitan,4019.000
6,Abbotsford,55a Park St,4,h,1600000.000,VB,Nelson,4/06/2016,2.500,3067.000,...,1.000,2.000,120.000,142.000,2014.000,Yarra City Council,-37.807,144.994,Northern Metropolitan,4019.000


* Create list of columns to drop.

* I keep only those things directly relevant to the attributes of the house or land.

* Distance to Central Business District (C.B.D.) is also kept since it's a numeric variable.

In [23]:
drop_list = ["Suburb","Address","Bedroom2", "Price","SellerG","Date","Postcode", "YearBuilt","CouncilArea","Lattitude",
             "Longtitude","Regionname","Propertycount"]
X = houses.drop(drop_list,1)
y = houses.Price

In [24]:
X.head()

,Rooms,Type,Method,Distance,Bathroom,Car,Landsize,BuildingArea
1,2,h,S,2.500,1.000,1.000,202.000,nan
2,2,h,S,2.500,1.000,0.000,156.000,79.000
4,3,h,SP,2.500,2.000,0.000,134.000,150.000
5,3,h,PI,2.500,2.000,1.000,94.000,nan
6,4,h,VB,2.500,1.000,2.000,120.000,142.000


In [25]:
X = X.fillna(X.mean())

In [26]:
X = pd.get_dummies(X)

In [27]:
X.head()

,Rooms,Distance,Bathroom,Car,Landsize,BuildingArea,Type_h,Type_t,Type_u,Method_PI,Method_S,Method_SA,Method_SP,Method_VB
1,2,2.500,1.000,1.000,202.000,156.835,1,0,0,0,1,0,0,0
2,2,2.500,1.000,0.000,156.000,79.000,1,0,0,0,1,0,0,0
4,3,2.500,2.000,0.000,134.000,150.000,1,0,0,0,0,0,1,0
5,3,2.500,2.000,1.000,94.000,156.835,1,0,0,1,0,0,0,0
6,4,2.500,1.000,2.000,120.000,142.000,1,0,0,0,0,0,0,1


In [28]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
#Y = data['Sales'].values.reshape(-1, 1)
#X = data[['TV','Radio','Newspaper']]
regr.fit(X, y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, y))


Coefficients: 
 [ 2.14281635e+05 -3.73029438e+04  2.19079675e+05  2.75918700e+04
  4.27086058e+00  5.37328089e+01  2.06413878e+05 -4.75804297e+04
 -1.58833448e+05 -1.39889693e+04  1.93199213e+04  2.18758532e+04
 -8.62246983e+04  5.90178931e+04]

Intercept: 
 316970.7645825241

R-squared:
0.4159905885904384


## Results ##

* Base value of a home in Melbourne is A$ 316,970.

* Each Room is worth ~ A$ 214,281.

* Distance from C.B.D. impacts value by A$ 37,302 per km.

* Each Bathroom is worth A$ 219079.

* Each Carspot is worth A$ 27,591.

* Each Sq. Meter of land is worth A$ 4.27.

* Building Area is worth A$ 53.73.

* An actual house is worth A$ 206,413 additionally to base value.

* A townhouse is worth an A$ 47,580 less than base value.

* An Apartment unit is worth A$ 158.833 less than base value.

* The following methods of sale impact price as follows from base value.
  1. PI (property passed in) -A$ 13,988.
  
  2. S (property sold) +A$ 19,319.
  
  3. SA (Sold Auction) +A$ 21,875
  
  4. SP (Sold Prior) -A$ 86,224
  
  5. VB (Vendor Bid) +A$ 59,017
  